In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
OUTPUT_DIR = './tmp/models'
datadir = './data'
is_tensorboard = False

In [3]:
datadir


'./data'

In [4]:

#filename=datadir+'/arr_keras_X_y_train_test.dump'


In [5]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
Dimensions= ((414, 224, 224, 3), (47, 224, 224, 3), (414,), (47,))
Number of classes= 3


In [6]:
cnn_layer_config={'feature_map_size':[128,]}
list_cnn_layer_filter = list(cnn_layer_config.values())[0]
list_cnn_layer_filter

[128]

In [7]:
import tensorflow as tf
import p8_util
import shutil

    
LEARNING_RATE = 0.003  #@param {type:"number"}
LEARNING_RATE = 0.03  #@param {type:"number"}

TRAIN_STEPS = 5  #@param {type:"integer"}
BATCH_SIZE = 49  #@param {type:"integer"}

ADANET_ITERATIONS = 2  #@param {type:"integer"}
NN_TYPE = 'DNN'
learn_mixture_weights=True

my_feature_columns, loss_reduction, tf_head \
= p8_util.get_tf_head("images",tuple_dimension, nClasses)

# A `Head` instance defines the loss function and metrics for `Estimators`.
# Tells Tensorfow how to compute loss function and metrics
#head = tf.contrib.estimator.multi_class_head(nClasses\
#                                             , loss_reduction=loss_reduction)
cnn_layer_config={'feature_map_size':[128]}
#cnn_layer_config=None

initialiser_name = 'xavier'
# When None then HE normal initialization takes place
initialiser_name = None
estimator = adanet.Estimator(
    head=tf_head,
    
    subnetwork_generator=p8_util.MyGenerator(
        nClasses,
        feature_columns=my_feature_columns,
        optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
        seed=p8_util.RANDOM_SEED,
        nn_type=NN_TYPE,
        dropout=0.5,
        learn_mixture_weights=False,
        layer_size=10,
        cnn_layer_config = cnn_layer_config,
        is_cnn_batch_norm = False,
        initializer=initialiser_name),
            
    adanet_lambda=.015,
    max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , tuple_dimension=tuple_dimension\
                          , training=False\
                          , batch_size=BATCH_SIZE),
        steps=None),
    config=p8_util.make_config(NN_TYPE,output_dir=OUTPUT_DIR))

*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/models/DNN', '_tf_random_seed': 42, '_save_summary_steps': 100000, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1be166bd30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
import time
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train", x_train, y_train\
                           , tuple_dimension=tuple_dimension\
                           , training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test", x_test, y_test\
                           , tuple_dimension=tuple_dimension\
                           , training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 100000 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0
INFO:tensorflow:Calling model_fn.

*** +++ generate_candidates() : Layer= 0


*** _NNAdaNetBuilder : Classes=3

*** _NNAdaNetBuilder : Classes=3
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'DNN_linear'


*** build_subnetwork() : features shape= (?, 224, 224, 3)
**** *** _build_dnn_subnetwork : Layers= 0

Instructions for updating:
`persisted_tensors` is deprecated, please use `shared` instead.

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_

INFO:tensorflow:Warm-starting variable: global_step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/logits/mixture_weight; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/

INFO:tensorflow:Saving candidate 't1_DNN_layer_1' dict for global step 4: accuracy/adanet/adanet_weighted_ensemble = 0.31914893, accuracy/adanet/subnetwork = 0.31914893, accuracy/adanet/uniform_average_ensemble = 0.31914893, architecture/adanet/ensembles = b'\no\n>adanet/iteration_1/ensemble_t1_DNN_layer_1/architecture/adanetB#\x08\x07\x12\x00B\x1d| DNN_layer_1 | DNN_layer_1 |J\x08\n\x06\n\x04text', average_loss/adanet/adanet_weighted_ensemble = 11.68587, average_loss/adanet/subnetwork = 11.68587, average_loss/adanet/uniform_average_ensemble = 11.68587, loss/adanet/adanet_weighted_ensemble = 11.68587, loss/adanet/subnetwork = 11.68587, loss/adanet/uniform_average_ensemble = 11.68587
INFO:tensorflow:Saving candidate 't1_DNN_layer_2' dict for global step 4: accuracy/adanet/adanet_weighted_ensemble = 0.31914893, accuracy/adanet/subnetwork = 0.3617021, accuracy/adanet/uniform_average_ensemble = 0.31914893, architecture/adanet/ensembles = b'\no\n>adanet/iteration_1/ensemble_t1_DNN_layer_2/a

INFO:tensorflow:Warm-starting variable: adanet/iteration_1/candidate_t0_DNN_layer_1/adanet/iteration_1/candidate_t0_DNN_layer_1/adanet_loss/local_step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_0/logits/mixture_weight; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_1/subnetwork/dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_1/subnetwork/dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_1/subnetwork/dense_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_1/subnetwork/dense_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/ite


*** +++ generate_candidates() : Layer= Tensor("adanet/iteration_1/ensemble_t1_DNN_layer_2/weighted_subnetwork_1/subnetwork/Const:0", shape=(), dtype=int64)


*** _NNAdaNetBuilder : Classes=3

*** _NNAdaNetBuilder : Classes=3
INFO:tensorflow:Building iteration 2
INFO:tensorflow:Building subnetwork 'DNN_layer_2'


*** build_subnetwork() : features shape= (?, 224, 224, 3)

**** *** _build_dnn_subnetwork : Layer= 0 / Layers= 2

**** *** _build_dnn_subnetwork : Layer= 1 / Layers= 2
**** *** _build_dnn_subnetwork : Layers= 2


+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3

+++ _check_logits_final_dim() : expected_logits_dimension= 3
INFO:tensorflow:Building subnetwork 'DNN_layer_3'


*** build_subnetwork() : features shape= (?, 224, 224, 3)

**** *** _build_dnn_subnetwork : Layer= 0 / Layers= 3

**** *** _build_dnn_subnetwork : Layer= 1 / Layers= 3

**** *** _build_dnn_subnetwork : Layer= 2 / Layers= 3
**** *** _bu

In [9]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 44.34582543373108
ACCURACY: 0.3191489279270172
ACCURACY/ADANET/ADANET_WEIGHTED_ENSEMBLE: 0.3191489279270172
ACCURACY/ADANET/SUBNETWORK: 0.3404255211353302
ACCURACY/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 0.3191489279270172
ARCHITECTURE/ADANET/ENSEMBLES: b'\n}\n>adanet/iteration_2/ensemble_t2_DNN_layer_3/architecture/adanetB1\x08\x07\x12\x00B+| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |J\x08\n\x06\n\x04text'
AVERAGE_LOSS: 3.7545225620269775
AVERAGE_LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 3.7545225620269775
AVERAGE_LOSS/ADANET/SUBNETWORK: 1.448772668838501
AVERAGE_LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 3.7545228004455566
LOSS: 3.7545225620269775
LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 3.7545225620269775
LOSS/ADANET/SUBNETWORK: 1.448772668838501
LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 3.7545228004455566
GLOBAL_STEP: 5


In [10]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [11]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS

#filenale_model_check = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-3'
#print_tensors_in_checkpoint_file(file_name=filenale_model_check, tensor_name='', all_tensors=False)
print("\n----------\n")
filename_model_checkpoint = './tmp/models/'+str(NN_TYPE)+'/model.ckpt-'+str(TRAIN_STEPS)
print_tensors_in_checkpoint_file(file_name=filename_model_checkpoint\
                                 , tensor_name='', all_tensors=False)



----------

adanet/iteration_0/candidate_t0_DNN_layer_1/adanet/iteration_0/candidate_t0_DNN_layer_1/adanet_loss/biased (DT_FLOAT) []
adanet/iteration_0/candidate_t0_DNN_layer_1/adanet/iteration_0/candidate_t0_DNN_layer_1/adanet_loss/local_step (DT_FLOAT) []
adanet/iteration_0/candidate_t0_DNN_layer_1/adanet_loss (DT_FLOAT) []
adanet/iteration_0/ensemble_t0_DNN_layer_1/bias (DT_FLOAT) [3]
adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/logits/mixture_weight (DT_FLOAT) []
adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense/bias (DT_FLOAT) [10]
adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense/kernel (DT_FLOAT) [150528,10]
adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense_1/bias (DT_FLOAT) [3]
adanet/iteration_0/ensemble_t0_DNN_layer_1/weighted_subnetwork_0/subnetwork/dense_1/kernel (DT_FLOAT) [10,3]
adanet/iteration_0/step (DT_INT64) []
adanet/iteration_0/train_op/is_over/is_ov

In [12]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [13]:
print("Architecture: {} / Accuracy= {} ".format(ensemble_architecture(results)\
                                                  , results['accuracy']))

Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' / Accuracy= 0.3191489279270172 


In [14]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNN_layer_1 | CNN_layer_2 |' : accuracy = 34% / Loss =  1.1142968

In [15]:
#import keras
#help(keras.layers.drop)

In [16]:
results.keys()
#is_tensorboard = False

dict_keys(['accuracy', 'accuracy/adanet/adanet_weighted_ensemble', 'accuracy/adanet/subnetwork', 'accuracy/adanet/uniform_average_ensemble', 'architecture/adanet/ensembles', 'average_loss', 'average_loss/adanet/adanet_weighted_ensemble', 'average_loss/adanet/subnetwork', 'average_loss/adanet/uniform_average_ensemble', 'loss', 'loss/adanet/adanet_weighted_ensemble', 'loss/adanet/subnetwork', 'loss/adanet/uniform_average_ensemble', 'global_step'])

In [17]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [18]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

In [19]:
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill